In [7]:
import random
import math
import pickle


class Neu:
    """
    Hammerstein-Wiener Recurrent Neural Network
    """

    def __init__(self, seg, input_n):
        self.segment = seg

        self.inputnumber = input_n
        self.hidnumber = 5
        self.outputnumber = 1

        self.epoch = 100
        self.learnspeedwd = 0.05  # 0.05
        self.learnspeedab = 0.0005  # 0.0005

        self.mse = 0

        self.X = []  # clear every time
        self.op = []  # clear every time

        self.w = []  # keep
        self.d1 = []  # keep
        self.fq = []  # clear every time
        self.oq = []  # clear every time

        self.BB = []  # keep
        self.AA = []  # keep
        self.fr = []  # clear every time
        self.orr = []  # clear every time

        self.CC = []  # keep
        self.d2 = []  # keep
        self.fs = []  # clear every time
        self.os = []  # clear every time

        self.Y = []  # clear every time

        self.X_temp = []  # clear every time
        self.BB_temp = []  # clear every time
        self.CC_temp = []  # clear every time
        self.data_s = []  # clear every time
        self.data_r = []  # clear every time
        self.data_q = []  # clear every time
        self.error = []  # keep
        self.count = 0  # keep
        self.training_MSE = []  # keep
        self.square_e = []  # keep
        self.OD_or_a_plus = []  # keep
        self.OD_or_b_plus = []  # keep

        self.createnetwork()

    def saveneu(self):
        """
        write neu to file
        """
        neuname = self.segment + '-' + self.daynumber + '.data'
        f = open(neuname, 'wb')
        # dump the object to a file
        pickle.dump(self, f)
        f.close()

    def createzero(self, createlist, row, col):
        """
        create zero list
        """
        createlist = []
        for i in range(row):
            temp = []
            for j in range(col):
                temp.append(0)
            createlist.append(temp)
        return createlist

    def createrandom(self, createlist, row, col):
        """
        create random list
        """
        createlist = []
        for i in range(row):
            temp = []
            for j in range(col):
                temp2 = random.uniform(-1, 1)
                temp.append(temp2)
            createlist.append(temp)
        return createlist

    def createnetwork(self):
        """
        create the network structure
        """
        self.AA = self.createrandom(self.AA, self.hidnumber, self.hidnumber)
        self.BB = self.createrandom(self.BB, self.hidnumber, 1)
        self.CC = self.createrandom(self.CC, self.outputnumber, self.hidnumber)
        self.w = self.createrandom(self.w, self.hidnumber, self.inputnumber)
        self.d1 = self.createrandom(self.d1, self.hidnumber, 1)
        self.d2 = self.createrandom(self.d2, self.outputnumber, 1)
        self.X = self.createzero(self.X, self.hidnumber, 1)
        self.Y = self.createzero(self.Y, self.outputnumber, 0)
        self.op = self.createzero(self.op, self.inputnumber, 1)
        self.fq = self.createzero(self.fq, self.hidnumber, 1)
        self.oq = self.createzero(self.oq, self.hidnumber, 1)
        self.fr = self.createzero(self.fr, self.hidnumber, 1)
        self.orr = self.createzero(self.orr, self.hidnumber, 1)
        self.fs = self.createzero(self.fs, self.outputnumber, 1)
        self.os = self.createzero(self.os, self.outputnumber, 1)
        self.X_temp = self.createzero(self.X_temp, self.hidnumber, 1)
        self.BB_temp = self.createzero(self.BB_temp, self.hidnumber, 1)
        self.CC_temp = self.createzero(self.CC_temp, self.outputnumber, self.hidnumber)
        self.data_s = self.createzero(self.data_s, self.outputnumber, 1)
        self.data_r = self.createzero(self.data_r, self.hidnumber, 1)
        self.data_q = self.createzero(self.data_q, self.hidnumber, 1)
        self.error = self.createzero(self.error, self.outputnumber, 1)
        self.training_MSE = self.createzero(self.training_MSE, self.outputnumber, 0)
        self.square_e = self.createzero(self.square_e, self.outputnumber, 1)
        self.OD_or_a_plus = self.createzero(self.OD_or_a_plus, self.hidnumber, self.hidnumber)
        self.OD_or_b_plus = self.createzero(self.OD_or_b_plus, self.hidnumber, 1)

    def cleartemporaltrain(self):
        """
        clear temp data every time
        """
        self.X_temp = self.createzero(self.X_temp, self.hidnumber, 1)
        self.BB_temp = self.createzero(self.BB_temp, self.hidnumber, 1)
        self.CC_temp = self.createzero(self.CC_temp, self.outputnumber, self.hidnumber)
        self.data_s = self.createzero(self.data_s, self.outputnumber, 1)
        self.data_r = self.createzero(self.data_r, self.hidnumber, 1)
        self.data_q = self.createzero(self.data_q, self.hidnumber, 1)
        self.X = self.createzero(self.X, self.hidnumber, 1)
        self.op = self.createzero(self.op, self.inputnumber, 1)
        self.fq = self.createzero(self.fq, self.hidnumber, 1)
        self.oq = self.createzero(self.oq, self.hidnumber, 1)

        self.fr = self.createzero(self.fr, self.hidnumber, 1)
        self.fs = self.createzero(self.fs, self.outputnumber, 1)
        self.os = self.createzero(self.os, self.outputnumber, 1)
        self.Y = self.createzero(self.Y, self.outputnumber, 0)

    def cleartemporalepoch(self):
        """
        clear temp data every epoch
        """
        self.orr = self.createzero(self.orr, self.hidnumber, 1)

        self.OD_or_a_plus = self.createzero(self.OD_or_a_plus, self.hidnumber, self.hidnumber)
        self.OD_or_b_plus = self.createzero(self.OD_or_b_plus, self.hidnumber, 1)

    def forward(self, inputlist):
        """
        calculate result
        """
        self.cleartemporaltrain()

        # layer 1: input layer
        for i in range(self.inputnumber):
            self.op[i][0] = inputlist[i]
            # print('op: ', self.op[i][0])

        # layer 2: hidden layer
        for i in range(self.hidnumber):
            for j in range(self.inputnumber):
                self.fq[i][0] = self.fq[i][0] + self.w[i][j] * self.op[j][0]
                # print('w: ', self.w[i][j])
            self.fq[i][0] = self.fq[i][0] + self.d1[i][0]
            # print('d1: ', self.d1[i][0])
            # print('fq: ', self.fq[i][0])

        for i in range(self.hidnumber):
            upper = math.exp(self.fq[i][0]) - math.exp(-self.fq[i][0])
            down = math.exp(self.fq[i][0]) + math.exp(-self.fq[i][0])
            self.oq[i][0] = upper / down
            # print('oq: ', self.oq[i][0])

        # layer 3: dynamic layer
        for i in range(self.hidnumber):
            self.X_temp[i][0] = self.orr[i][0]

        temp = []
        for i in range(self.hidnumber):
            temp.append(self.BB[i][0] * self.oq[i][0])
            # print('temp: ', temp[i])

        for i in range(self.hidnumber):
            for j in range(self.hidnumber):
                self.fr[i][0] = self.fr[i][0] + self.AA[i][j] * self.orr[j][0]
            self.fr[i][0] = self.fr[i][0] + temp[i]
            # print('fr: ', self.fr[i][0])

        for i in range(self.hidnumber):
            self.orr[i][0] = self.fr[i][0]

        # layer 4: output layer
        for i in range(self.outputnumber):
            for j in range(self.hidnumber):
                self.fs[i][0] = self.fs[i][0] + self.CC[i][j] * self.orr[j][0]
            self.fs[i][0] = self.fs[i][0] + self.d2[i][0]
            # print(self.fs[i][0])

        for i in range(self.outputnumber):
            self.os[i][0] = self.fs[i][0]

        for i in range(self.outputnumber):
            self.Y[i] = self.os[i][0]

        return self.Y

    def backward(self, expect):
        """
        adjust weight
        """
        # decide error
        for i in range(self.outputnumber):
            self.error[i][0] = expect[i] - self.Y[i]
            # print('error: ', self.error)

        # calculate MSE
        self.count = self.count + 1
        for i in range(self.outputnumber):
            self.square_e[i][0] = self.square_e[i][0] + self.error[i][0] * self.error[i][0]
            self.training_MSE[i].append(self.square_e[i][0] / self.count)
            # print('training_MSE', self.training_MSE[i])

        for i in range(self.outputnumber):
            self.data_s[i][0] = self.error[i][0]
            # print('data_s: ', self.data_s[i][0])

        # update d2
        for i in range(self.outputnumber):
            self.d2[i][0] = self.d2[i][0] + self.learnspeedwd * self.data_s[i][0]
            # print('d2: ', self.d2[i][0])

        # update CC
        for i in range(self.outputnumber):
            for j in range(self.hidnumber):
                self.CC_temp[i][j] = self.CC[i][j]

        for i in range(self.outputnumber):
            for j in range(self.hidnumber):
                self.CC[i][j] = self.CC[i][j] + self.learnspeedab * self.data_s[i][0] * self.orr[j][0]
                # print('CC: ', self.CC[i])

        for i in range(self.outputnumber):
            for j in range(self.hidnumber):
                self.data_r[j][0] = self.data_s[i][0] * self.CC_temp[i][j]
                # print('data_r: ', self.data_r[j][0])

        # update BB
        for i in range(self.hidnumber):
            self.BB_temp[i][0] = self.BB[i][0]

        for i in range(self.hidnumber):
            self.OD_or_b_plus[i][0] = self.oq[i][0] + self.AA[i][i] * self.OD_or_b_plus[i][0]
            self.BB[i][0] = self.BB[i][0] + self.learnspeedab * self.data_r[i][0] * self.OD_or_b_plus[i][0]
            # print('BB: ', self.BB[i][0])
            # print('OD_or_b_plus: ', self.OD_or_b_plus[i][0])

        # update AA
        for i in range(self.hidnumber):
            for j in range(self.hidnumber):
                self.OD_or_a_plus[i][j] = self.X_temp[j][0] + self.AA[i][i] * self.OD_or_a_plus[i][j]
                self.AA[i][j] = self.AA[i][j] + self.learnspeedab * self.data_r[i][0] * self.OD_or_a_plus[i][j]
                # print('AA: ', self.AA[i][j])
                # print('OD_or_a_plus: ', self.OD_or_a_plus[i][j])

        for i in range(self.hidnumber):
            cos = math.exp(self.fq[i][0]) + math.exp(-self.fq[i][0])
            self.data_q[i][0] = self.data_r[i][0] * self.BB_temp[i][0] * 4 / (cos * cos)
            # print('data_q: ', self.data_q[i][0])

        # update d1
        for i in range(self.hidnumber):
            self.d1[i][0] = self.d1[i][0] + self.learnspeedwd * self.data_q[i][0]
            # print('d1: ', self.d1[i][0])

        # update w
        for i in range(self.hidnumber):
            for j in range(self.inputnumber):
                self.w[i][j] = self.w[i][j] + self.learnspeedwd * self.data_q[i][0] * self.op[j][0]
                # print('w: ', self.w[i][0])

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import csv
import math
from multiprocessing import Pool
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

In [3]:
# LSTM 臺大資訊大樓 (有雨)

monthOf2015 = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
ntu = []
rain = []

count = 0
f1 = open('raindata.csv', 'r', encoding='utf8')
while True:
    line1 = f1.readline().strip()
    if line1 == '':
        break
    line1List = line1.split(',')
    if line1[-1] == 'T':
        for i in range(0, 24):
            rain.append(float(line1List[i+1]))
        count+=1
        f2 = open('day/' + str(line1[0:4]) + '.csv', 'r', encoding='utf8')
        while True:
            line2 = f2.readline().strip()
            if line2 == '':
                break
            line2List = line2.split(',')
            if line2List[0] == '臺大資訊大樓':
                for i in range(0, 24):
                    ntu.append(int(line2List[i+1]))
                break
        f2.close()
f1.close()
print(count)

95


In [4]:
df_ubike = pd.DataFrame({'ubike':ntu})
print(df_ubike.describe())

df_rain = pd.DataFrame({'rain':rain})
print(df_rain.describe())

             ubike
count  2280.000000
mean     23.261842
std      26.791368
min       0.000000
25%       3.000000
50%      15.000000
75%      33.000000
max     176.000000
              rain
count  2280.000000
mean      1.080570
std       4.040805
min       0.000000
25%       0.000000
50%       0.000000
75%       0.500000
max      50.500000


In [5]:
sc = MinMaxScaler(feature_range=(-0.9, 0.9))
sc.fit(df_ubike)
df_ubike_std = sc.transform(df_ubike)

sc = MinMaxScaler(feature_range=(-0.9, 0.9))
sc.fit(df_rain)
df_rain_std = sc.transform(df_rain)

In [6]:
df_rnn1 = pd.DataFrame(columns=['Rain','X1'])

for i in range(len(df_ubike)):
    df_rnn1.loc[i,'Rain'] = np.asscalar(df_rain_std[i])
    df_rnn1.loc[i,'X1'] = np.asscalar(df_ubike_std[i])

df_rnn1

,Rain,X1
0,-0.882178,-0.838636
1,-0.9,-0.879545
2,-0.810891,-0.9
3,-0.864356,-0.889773
4,-0.9,-0.889773
5,-0.882178,-0.9
6,-0.9,-0.879545
7,-0.9,-0.736364
8,-0.9,-0.7875
9,-0.882178,-0.715909


In [17]:
for day in [1,2,3,4,5,6]:
    # Type 1 Neural Network---------------------------------------------------------------
    point = 0  # 先跳過有問題那幾筆
    cases = []
    labels = []
    # day = 1
    for i in range(0, 1824):
        one_day = list(df_rnn1.iloc[point:point + day, 1]) + list(df_rnn1.iloc[point+day-1:point+day, 0])
#         print(one_day)
        cases.append(one_day)
        labels.append([df_rnn1.iloc[point + day, 1]])
#         print(df_rnn1.iloc[point + day, 1])
        point += 1
#     print(point)
    cases_test = []
    labels_test = []
    for j in range(1824, 2279-day):
        one_day = list(df_rnn1.iloc[point:point + day, 1]) + list(df_rnn1.iloc[point+day-1:point + day, 0])
        cases_test.append(one_day)
        labels_test.append([df_rnn1.iloc[point + day, 1]])
        point += 1
    # ---------------------------------------------------------------------------------------
    mse_all = []
    total_start_time = time.time()
    for r in range(10):
        start = time.time()
        neutest = Neu('03F2614S-03F2709S' + ' #' + str(r), day+1)
        print(day+1)
        trainfinished = False
        while trainfinished != True:
            try:
                for i in range(neutest.epoch):
                    # print(i + 1)
                    # count = 0
                    for c, l in zip(cases, labels):
                        inputlist = c
                        expect = l

                        # print('input: ', inputlist)
                        # print('expect: ', expect)

                        result = neutest.forward(inputlist)
                        # print('result: ', result)
                        neutest.backward(expect)
                        # print(count)
                        # count = count + 1
                        # if count == 288:
                        #     count = 0
                    neutest.cleartemporalepoch()
            except OverflowError:
                neutest = Neu('03F2614S-03F2709S' + ' #' + str(r), day+1)
                print('math error')
                print(day+1)
            else:
                trainfinished = True
        end = time.time()
        elapsed = end - start
        print("Time taken: ", elapsed, "seconds.")
        # test
        predict_all = []
        for test1 in cases_test:
            result = neutest.forward(test1)
            predict_all.append(result)
        mse = mean_squared_error(labels_test[:], predict_all[:])
        print('MSE :', mse)
        mse_all.append(mse)

    total_end_time = time.time()
    print("Total time taken: ", total_end_time - total_start_time, "seconds.")

    print(mse_all)
    print(np.array(mse_all).min())


    plt.plot(labels_test, 'b')
    plt.plot(predict_all, 'r')
    plt.show()
    print("---------------------------------------------------------")

2
Time taken:  23.626139879226685 seconds.
MSE : 0.0294946439152
2
math error
2
math error
2
math error
2
math error
2
Time taken:  24.248692989349365 seconds.
MSE : 0.030164180505
2
math error
2
math error
2
Time taken:  23.511423110961914 seconds.
MSE : 0.0291909999996
2
Time taken:  23.70565629005432 seconds.
MSE : 0.0300063622222
2
Time taken:  23.42532706260681 seconds.
MSE : 0.0331507906217
2
math error
2
math error
2
math error
2
math error
2
math error
2
Time taken:  23.391923904418945 seconds.
MSE : 0.0303877538779
2
math error
2
math error
2
Time taken:  23.23124098777771 seconds.
MSE : 0.0293401038532
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
Time taken:  23.37035608291626 seconds.
MSE : 0.0308533012759
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
math error
2
ma

KeyboardInterrupt: 

In [60]:
len(labels)

1824

In [59]:
len(labels_test)

454

In [56]:
df_rnn1[1284:1290]

,Rain,X1
1284,-0.9,-0.265909
1285,-0.9,-0.276136
1286,-0.871485,-0.5625
1287,-0.775248,-0.695455
1288,-0.9,-0.286364
1289,-0.9,0.347727
